In [29]:
import os 

from dotenv import load_dotenv
load_dotenv()


OLLAMA_URL=os.getenv("OLLAMA_URL")
LLM_MODEL=os.getenv("LLM_MODEL")
LLM_MODEL_VISUAL=os.getenv("LLM_MODEL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
DATAPATH=os.getenv("DATAPATH")
#OLLAMA_URL='http://localhost:11434'
#LLM_MODEL="llama3.2:3b" # max retires
#LLM_MODEL="gemma3:4b" # no tool support
LLM_MODEL_VISUAL="gemma3:4b"
LLM_MODEL="llama3.2:1b"
DATAPATH="/app/data"
#LLM_MODEL="llava:7b"
#LLM_MODEL="codellama:7b"
#LLM_MODEL="qwen2.5-coder:1.5b" # no tool support
#LLM_MODEL="deepseek-r1:1.5b" # does not support tools
#os.setenv("no_proxy=localhost,127.0.0.1,*.my-it-solutions.net,ollama")

print(OLLAMA_URL)
print(f"model: {LLM_MODEL}")
print(f"visual model: {LLM_MODEL_VISUAL}")
print(f"embedding: {EMBEDDING_MODEL}")
print(f"datapath {DATAPATH}")
print(f"no_proxy {os.getenv('no_proxy')}")

http://ollama:11434
model: llama3.2:1b
visual model: gemma3:4b
embedding: nomic-embed-text:latest
datapath /app/data
no_proxy 127.0.0.0,localhsot,ollama,notebook


Test connection to ollama

In [2]:
import requests
import time

response = requests.get(OLLAMA_URL,timeout=3)
if response.status_code == requests.codes.ok:
    print(response)
else:
    print(response)    

<Response [200]>


Initialize LLM

In [30]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model=LLM_MODEL, request_timeout=120.0, base_url=OLLAMA_URL, temperature=0.0)



In [31]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, PromptTemplate
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(
    input_files=[f"{DATAPATH}/testdata.txt",f"{DATAPATH}/additionalinfo.txt"]
).load_data()
print("loaded")

Settings.embed_model = OllamaEmbedding(model_name=EMBEDDING_MODEL, base_url=OLLAMA_URL,embed_batch_size=100)
Settings.text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)
# ollama
Settings.llm = llm

index = VectorStoreIndex.from_documents(
documents,
#embed_model=ollama_embedding,
)
print("finish")

loaded


/opt/venv/lib/python3.13/site-packages/pydantic/_internal/_core_utils.py:448: RuntimeWarning: coroutine 'Agent.run' was never awaited
  def collect_refs(s: core_schema.CoreSchema, recurse: Recurse) -> core_schema.CoreSchema:


finish


Demo data

In [33]:
messages = [{ "From": "Marco.Polo@wtc.com<Marco Polo>", "To": "Boris@arm.ru<Boris Invanovic>", "Subject": "Lieferung", 
            "Content": "Eine neue Lieferung steht bereit. Wir senden sie morge mit dem Flug TK9518 nach BEG"}
            ,{ "From": "Boris@arm.ru<Boris Invanovic>", "To": "Chef51@mega.de", "Subject": "Info", 
            "Content": "Morgen können die neuen Bananen in BEG abgeholt werden. Grüsse Bo"} #nick name Bo
            ,{ "From": "Boris@arm.ru<Boris Invanovic>", "To": "agent51@bunny.com", "Subject": "FW: Info", 
            "Content": "FYI\n >>Morgen können die neuen Bananen in BEG abgeholt werden. Grüsse Bo"} #nick name Bo
]


Extract Entities

In [34]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Person(BaseModel):
    """Person entity."""
    name: str = Field(
        ..., description="The name of a person"
    )
    nickname: str = Field(
        ..., description="The nickname of a person"
    )
    email: str = Field(
        ..., description="The email address of a person"
    )    

class PersonList(BaseModel):
    persons: List[Person]
    

def add_person_list(new_persons: PersonList, existing_list: Optional[PersonList] = None) -> PersonList:
    """
    Add a person list to an empty or existing array of persons.
    
    Args:
        new_persons: A PersonList object containing persons to be added
        existing_list: An optional existing PersonList object to add to
    
    Returns:
        A new PersonList containing the merged list of persons
    """
    if existing_list is None:
        return new_persons
    
    # Create a new list with all persons from both lists
    merged_persons = PersonList(persons=existing_list.persons + new_persons.persons)
    return merged_persons   
    

In [21]:
# Knowledge base
import json
knowledge_person = []
knowledge_person_plain = []

#add_person_list(PersonList(Person=[{"name":"Mario","nickname":"MarioBo","email":"ma@acme.com"}]).Person,knowledge_person)

people = PersonList(persons=[
    {"name": "John", "nickname": "Johnny", "email": "john@example.com"},
    {"name": "Jane", "nickname": "Janie", "email": "jane@example.com"},
    {"name": "Boris", "nickname": "Invanovic", "email": "agent51@bunny.com"},
])
print(people)
                    
person = Person(
    name="John Doe",
    nickname="Johnny",
    email="john@example.com"
)
print(person)
json_data = '{"name": "Bob Johnson", "nickname": "Bobby", "email": "bob@example.com"}'
data_dict = json.loads(json_data)
person = Person.model_validate(data_dict)
print(person)

persons=[Person(name='John', nickname='Johnny', email='john@example.com'), Person(name='Jane', nickname='Janie', email='jane@example.com'), Person(name='Boris', nickname='Invanovic', email='agent51@bunny.com')]
name='John Doe' nickname='Johnny' email='john@example.com'
name='Bob Johnson' nickname='Bobby' email='bob@example.com'


Save new entities to memmory

In [35]:
#from llama_index.program.openai import OpenAIPydanticProgram
#from llama_index.core.extractors import PydanticProgramExtractor
import nest_asyncio

nest_asyncio.apply()

EXTRACT_TEMPLATE_STR = """\
Here is the content of the section:
----------------
{context_str}
----------------
Given the contextual information, extract out all person object.
A Person entity consists of: name, email.
Only use the context information. Do not assume any other facts.
Your output MUST follow this json EXACT format with EXACT field names:\n\n
[{"Persons:[{"name":"Person Name"},{"email":[Email address],"nickname":{Nickname}}}]}  ]
\
"""

EXTRACT_TEMPLATE_STR1 = """\
Given the contextual information, extract a list of persons.
If no persons are found return an emtpy list.
Only use the context information. Do not assume any other facts.
Your output MUST follow this json EXACT format with EXACT field names:\n\n
PersonList(persons=[
    {"name": "John", "nickname": "Johnny", "email": "john@example.com"},
    {"name": "Jane", "nickname": "Janie", "email": "jane@example.com"},
])
The output MUST be always a JSON Array.
Only respond in the correct format, do not include additional properties in the JSON.\n
"""
EXTRACT_TEMPLATE_STR2 = """\
your are a assistand to extract informaiton. If needed you use tools.
From the provided context, extract a list of persons mentioned in the text.

Each person MUST include:
"name": Full name of the person.
"nickname": Nickname if mentioned, otherwise use an empty string "".
"email": Email address if mentioned, otherwise use an empty string "".
Constraints:

Do NOT assume or invent any information not explicitly present in the context.
If no persons are found, return an empty list: PersonList(persons=[]).
Always respond with ONLY valid JSON following this EXACT structure (no extra text, no explanations):
json
Copy
Edit
PersonList(persons=[
    {"name": "John", "nickname": "Johnny", "email": "john@example.com"},
    {"name": "Jane", "nickname": "Janie", "email": "jane@example.com"}
])
All fields MUST be present for each person, even if empty.
The output MUST always be a valid JSON array inside PersonList, even when empty.
Context to analyze:
"""

# extract only entities
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel

ollama_model = OpenAIModel(model_name=LLM_MODEL, base_url=f"{OLLAMA_URL}/v1")
                     
#agent = Agent(model=ollama_model, result_type=str,system_prompt=EXTRACT_TEMPLATE_STR,retries=3)
# deps_tpye is the input 
#agent = Agent(model=ollama_model,result_type=PersonList,deps_type=str,system_prompt=EXTRACT_TEMPLATE_STR2,retries=4)
agent = Agent(model=ollama_model,result_type=str,deps_type=str,system_prompt=EXTRACT_TEMPLATE_STR2,retries=1)
#TODO

#import json
#PersonList_schema = PersonList.model_json_schema()  # (1)!
#print(json.dumps(PersonList_schema, indent=2))  # (2)!
#print(json.dump())
knowledge_person_plain = []
for message in messages:
    print(f"query: {message}")
    entity_result = agent.run_sync(f"{message}")
    print(entity_result.data) 
    #print(len(entity_result.data))
    if isinstance(entity_result, PersonList):
        add_person_list(knowledge_person.data)
    if len(entity_result.data) > 2:        
        knowledge_person_plain.append(entity_result.data)
        print(entity_result.data)
    print("***")

print("------------------")
print(knowledge_person_plain)
#print(entity_result.data)
#print(entity_result.usage())
#memory.save(org_result.data)

#print(entity_result.model_dump())

query: {'From': 'Marco.Polo@wtc.com<Marco Polo>', 'To': 'Boris@arm.ru<Boris Invanovic>', 'Subject': 'Lieferung', 'Content': 'Eine neue Lieferung steht bereit. Wir senden sie morge mit dem Flug TK9518 nach BEG'}
json
Edit
PersonList(persons=[
    {"name": "Marco Polo", "nickname": "", "email": "Marco.Polo@wtc.com"},
    {"name": "Boris Invanovic", "nickname": "Boris", "email": "Boris@arm.ru"}
])
json
Edit
PersonList(persons=[
    {"name": "Marco Polo", "nickname": "", "email": "Marco.Polo@wtc.com"},
    {"name": "Boris Invanovic", "nickname": "Boris", "email": "Boris@arm.ru"}
])
***
query: {'From': 'Boris@arm.ru<Boris Invanovic>', 'To': 'Chef51@mega.de', 'Subject': 'Info', 'Content': 'Morgen können die neuen Bananen in BEG abgeholt werden. Grüsse Bo'}
json
Copy
Edit
PersonList(persons=[
    {"name": "Boris", "nickname": "", "email": "Boris@arm.ru"},
    {"name": "Ivan", "nickname": "Petyr", "email": ""}
])
json
Copy
Edit
PersonList(persons=[
    {"name": "Boris", "nickname": "", "email"

Relation Agent

In [ ]:
# relation talk / known

RELATE_TEMPLATE_STR = f"""\
Task:
From the provided data, extract a graph of persons and their relationships.

Graph structure:

Nodes: Each person identified in the "Entities to analyze".
Edges: Represent relationships between persons based on message content.
Types of relations (edges):

"talk": Two persons talk directly to each other.
"known": The content indicates that one person knows another (e.g., mentions them as a known contact, friend, colleague).
Constraints and Output Rules:

ONLY use persons listed under "Entities to analyze" for nodes. Do not invent persons.
If no relations are found, return an empty "edges" list.
Always return a valid JSON object matching EXACTLY the structure shown in the example.
DO NOT add any explanatory text or comments. Only output the JSON.
All node IDs must match person names exactly as listed in "Entities to analyze" (case sensitive).

Example output:
{{
  "graph": {{
    "nodes": {{
      "Person1": {{"name": "John"}},
      "Person2": {{"name": "Alice"}}
    }},
    "edges": [
      {{
        "source": "Person1",
        "target": "Person2",
        "relation": "talk"
      }}
    ]
  }}
}}

Important:

Use exact names from "Entities to analyze" as node IDs.
Always include all persons as nodes, even if they have no edges.
Follow exact JSON field names and structure as shown.


Data to analyze:
"""

rel_agent = Agent(model=ollama_model,result_type=str,deps_type=str,system_prompt=RELATE_TEMPLATE_STR,retries=1)
knowledge_person_plain

relation_result = rel_agent.run_sync(f"""{messages} entities: {knowledge_person_plain}""")

try:    
    print(json.dumps(json.loads(relation_result.data), indent=4))
except Exception as X:
    print(f"error {X}, print unformated")
    print(relation_result)


In [1]:
import json
import networkx as nx
import matplotlib.pyplot as plt

graph_data = json.loads(relation_result.data)
# Your JSON data (stored in a variable directly for this example)
xjson_data = '''
{
    "graph": {
        "nodes": {
            "Marco Polo": {
                "name": "Marco Polo"
            },
            "Boris Invanovic": {
                "name": "Boris Invanovic"
            },
            "Bo": {
                "name": "Boris Invanovic"
            }
        },
        "edges": [
            {
                "source": "Marco Polo",
                "target": "Boris Invanovic",
                "relation": "talk"
            },
            {
                "source": "Boris Invanovic",
                "target": "Bo",
                "relation": "talk"
            },
            {
                "source": "Boris Invanovic",
                "target": "Chef51@mega.de",
                "relation": "known"
            }
        ]
    }
}
'''

# Parse JSON data
graph_data = json.loads(json_data)

# Create a NetworkX graph
G = nx.Graph()

# Add nodes - here's the fix: use the node_id directly as both the node and its label
for node_id, node_info in graph_data["graph"]["nodes"].items():
    G.add_node(node_id)  # The node_id is the key like "Marco Polo"

# Also add any nodes that appear in edges but not in nodes section
all_nodes = set(graph_data["graph"]["nodes"].keys())
for edge in graph_data["graph"]["edges"]:
    if edge["source"] not in all_nodes:
        G.add_node(edge["source"])
    if edge["target"] not in all_nodes:
        G.add_node(edge["target"])

# Add edges with relation as attribute
for edge in graph_data["graph"]["edges"]:
    G.add_edge(edge["source"], edge["target"], relation=edge["relation"])

# Draw the graph
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)  # seed for reproducibility

# Draw nodes and labels (directly use node IDs as labels)
nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=2000)
nx.draw_networkx_labels(G, pos, font_size=12)  # No need for a labels dict

# Draw edges
nx.draw_networkx_edges(G, pos, width=2, edge_color='gray')

# Draw edge labels (relation type)
edge_labels = nx.get_edge_attributes(G, 'relation')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=10)

plt.axis('off')
plt.title('Person Relation Graph', fontsize=14)
plt.show()

NameError: name 'relation_result' is not defined

In [2]:
from datetime import date
from pydantic_ai import RunContext

@agent.system_prompt
def add_the_date() -> str:  
    return f'The date is {date.today()}.'

result = agent.run_sync('What is the date?', deps='Frank')
print(result.data)

NameError: name 'agent' is not defined

Extract data from image (only works with a model supporting images)

In [9]:
import nest_asyncio
nest_asyncio.apply()

EXTRACT_IMG_TEMPLATE_STR = """\
Extract all possible information from the image.

{ctx.deps}

Add for any non latin text a translation to the latin alphabet in brackets.

A Person entity consists of: name, email, birthdate, nationality.
Only use the context information. Do not assume any other facts.
If it's the  passport or id card the natianlity might be the same as the issuer country.
Your output MUST follow this JSON EXACT format with EXACT field names:
{
  "persons": [
    {
      "name": "[Person Name]",    
      "email": "[Email address]",
      "birthdate": "[birthdate]",
      "nationality": "[nationality]"
    }
  ]
}
"""

import base64
# Create the agent instance
#agentImage = Agent(template=EXTRACT_IMG_TEMPLATE_STR)  # Define the agent
# extract only entities
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel

ollama_image_model = OpenAIModel(model_name=LLM_MODEL_VISUAL, base_url=f"{OLLAMA_URL}/v1",api_key="not-needed")
agentImage = Agent(model=ollama_image_model,result_type=str,deps_type=str,system_prompt=EXTRACT_IMG_TEMPLATE_STR,retries=1)

# Open the image file in binary mode
def load_img(path):
    with open(path, 'rb') as img_file:
        encoded_string = base64.b64encode(img_file.read())    
    return f'data:image/jpeg;base64,{encoded_string.decode("utf-8")}'

    
#image_data = load_img(f'{DATAPATH}/img/DO_UKR_AV.jpg')
image_data = load_img('/app/data/img/DO_UKR_AV.jpg')

print(f"image loaded: {image_data[:30]}...") 

from pydantic_ai import Agent, BinaryContent ,ImageUrl, RunContext
message_history=[]

result = agentImage.run_sync(
    [
        'Extrahiere die Informationen.',
        ImageUrl(url=image_data),
        
    ]
    ,message_history=[]
)
print(result.data)
#message_history = result.new_messages()
print(message_history)
print("new test")



image loaded: data:image/jpeg;base64,/9j/4gx...
```json
{
 "persons": [
   {
     "name": "МІКОЛАЙ",
     "email": "",
     "birthdate": "12 12 1989",
     "nationality": "УКРАЇНА/UKR"
   }
 ]
}
```
[]
new test


In [16]:
from pydantic.dataclasses import dataclass

image_data = load_img('/app/data/img/DO_UKR_AV.jpg')


@dataclass
class ImageExtarctorHelper:
    operator: str
    image_url: ImageUrl

@agentImage.system_prompt  
def add_the_users_name(ctx: RunContext[ImageExtarctorHelper]) -> str:
    #print(f"the operator name is {ctx.deps} The image is {ctx.deps}.")
    #print(ctx)
    return f"the operator name is  {ctx.operator} The image url is {ctx.image_url}." #ctx.image_url
#@support_agent.tool

#print(add_the_users_name(ImageExtarctorHelper(operator="Bob", image_url=ImageUrl(url=image_data))))

# deps = ImageExtarctorHelper(operator="Bob", image_url=image_data)
deps = ImageExtarctorHelper(operator="Bob", image_url=ImageUrl(url=image_data))
#BinaryContent(data=image_response.content, media_type='image/png') )
result = agentImage.run_sync('What my name?', deps=deps, 
                             #usage_limits=UsageLimits(response_tokens_limit=100),
                            )

print(result.data)

AttributeError: 'RunContext' object has no attribute 'operator'

GAP demmo

In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel, ConfigDict

class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def find_person(cls, *, name: str) -> Person | None:
        if name == 'John':
            return Person(name='John', nickname='Johnny', email='john@example.com')

    @classmethod
    async def add_person(cls, *, person: Person) -> Person | None:
        print(f"call add_person for {person}")
        knowledge_person.append(person)
        return true
            
print(people)
#print(f"found persons: {knowledge_person_plain}")

message_persons =  PersonList(persons=[
    {"name": "Marco Polo", "nickname": "", "email": "Marco.Polo@wtc.com"},
    {"name": "Jane", "nickname": "Janie", "email": "jane@example.com"},
    {"name": "Boris", "nickname": "Invanovic", "email": "agent51@bunny.com"},
])

knowledge_person = PersonList(persons=[
    {"name": "John", "nickname": "Johnny", "email": "john@example.com"},
    {"name": "Jane", "nickname": "Janie", "email": "jane@example.com"},
    {"name": "Boris", "nickname": "Invanovic", "email": ""},
])

GAP_IMG_TEMPLATE_STR="""
You are a carefull analyst. 
wheck wich person are already known and which person are new in the message person list.
Suggest the new persons, which could be added to the kwnon person list.

You MUST only use person names provieded by the lists. Do not assume anything not provided by the input.
"""

@dataclass
class SupportDependencies:
    model_config = ConfigDict(extra='ignore')
    name: str
    known_person: PersonList
    message_person: PersonList
    db: DatabaseConn

# This pydantic model defines the structure of the result returned by the agent.
class SupportResult(BaseModel):    
    known_persons: PersonList = Field(description='The persons already known')
    new_persons: PersonList = Field(description='The persons not yet known and candiates to add to the kwown person list')
    


#ollama_image_model = OpenAIModel(model_name=LLM_MODEL_VISUAL, base_url=f"{OLLAMA_URL}/v1",api_key="not-needed")
ollama_model = OpenAIModel(model_name=LLM_MODEL, base_url=f"{OLLAMA_URL}/v1")
#ollama_gap_model = OpenAIModel(model=ollama_model, base_url=f"{OLLAMA_URL}/v1",api_key="not-needed")

gap_agent = Agent(
    ollama_model,
    deps_type=SupportDependencies,
    # The response from the agent will, be guaranteed to be a SupportResult,
    # if validation fails the agent is prompted to try again.
    #result_type=SupportResult,
    result_type=str,
    system_prompt=GAP_IMG_TEMPLATE_STR,
    retries=10,
)

@gap_agent.tool
async def find_person(
    ctx: RunContext[SupportDependencies]
) -> float:
    """Returns if a Person if the person is found in the database."""
    # The docstring of a tool is also passed to the LLM as the description of the tool.
    # Parameter descriptions are extracted from the docstring and added to the parameter schema sent to the LLM.
    known_person = await ctx.deps.db.find_person(
        name=ctx.deps.name,
    )
    return known_person

@gap_agent.tool
async def add_person(
    tx: RunContext[SupportDependencies], new_person: Person
) -> float:
    """Add a person to the database."""
    # The docstring of a tool is also passed to the LLM as the description of the tool.
    # Parameter descriptions are extracted from the docstring and added to the parameter schema sent to the LLM.
    ctx.deps.known_person.append(new_person) 
    return "person added"   

deps = SupportDependencies(name="Frank",known_person=knowledge_person, message_person=message_persons, db=DatabaseConn())
result = await gap_agent.run('List all persons from the message_person list which are not known in the known_person list?', deps=deps)
print(result.data)

result = await gap_agent.run('call for each unknown person the fuction add_person to add them to the known person list?', deps=deps)
print(result.data)



persons=[Person(name='John', nickname='Johnny', email='john@example.com'), Person(name='Jane', nickname='Janie', email='jane@example.com'), Person(name='Boris', nickname='Invanovic', email='agent51@bunny.com')]


In [ ]:
from llama_index.core.tools import QueryEngineTool

query_engine = index.as_query_engine(similarity_top_k=3)

knowledge_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="knowledge_tool",
    description="""A RAG engine with some basic facts persons. Ask natural-language questions about persons and their properties and relations.
              if the knowledge_tool has no relatied information, ignore the answer.
              """
)

In [ ]:
from llama_index.core.tools import FunctionTool

# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
def find_person(name: str, **kwargs):
    """
    provides information about known persons. including ther detail information like birthdate

    args:
        name
    """
    # Mock response; replace with real query logic
    person_data = {
        "anna gölding": {"birthdate": "October 24, 1734", "known_for": "Last witch executed in Switzerland.","object_id":"1234","relations":{"knows other person":"ron paul","organzation":"pilz mafia"}},
        "john doe": {"birthdate": "Unknown", "known_for": "Placeholder name for anonymous individuals."},
        "ron paul": {"birthdate": "May 1, 1928", "known_for": "Talking a lot."},
        "miranda meyers": {"birthdate": "Aug 11, 1998", "known_for": "Miranda verkauft gerne verdorbens Eis. Das Eis erhält sie illegal von Litauen, wo es mit Mäusemilch hergestellt wird."},
    }
    return person_data.get(name.lower(), "No information available for this person.")

find_person_tool = FunctionTool.from_defaults(
    fn=find_person,
    name="find_person",
)


def find_organization(name: str, **kwargs):
    """
    provides information about known official and inofficial organzations.

    args:
        name
    """
    # Mock response; replace with real query logic
    org_data  = {
        "un": {"name": "United Nations", "description": "The Security Council has primary responsibility for the maintenance of international peace and security.","id":"200","relations":{""}},
        "pilz mafia": {"name": "Pilz Mafia", "description": "","id":"201","members":{"anna gölding","ron paul"}},
        "acme company": {"name":"acme company","description":"placeholder company"},
    }
    return org_data.get(name.lower(), "No information available for this organization.")

find_orgnization_tool = FunctionTool.from_defaults(
    fn=find_organization,
    name="find_organization",
)

#def get_messages(name: str, min_daterange: datetime, max_daterange: datetime):
def get_messages(name: str, min_daterange: str, max_daterange: str, **kwargs):
    """
    Retrieve information about communications between two or more people within a given date range.

    # Example usage:
        name = "c1"
        min_daterange = ISO8601 date string
        max_daterange = ISO8601 date string
        messages = get_messages(name, min_daterange, max_daterange)

    Args:
        name (str): The name of the context always use c1.
        min_daterange (datetime): The start of the date range.
        max_daterange (datetime): The end of the date range.

    Returns:
        dict: A dictionary of messages for the given name within the date range.
    """
    # Mock response; replace with real query logic
    min_daterange_ts = parser.parse(min_daterange)
    max_daterange_ts = parser.parse(max_daterange)
    org_data = {
        "c1_1738446338": {"sender": "Ron Paul", "message": "Anna Gölding ist gestorben.", "timestamp": datetime(2025, 1, 1)},
        "c1_1738446338": {"sender": "Pilz Mafia", "message": "Hat Sie mit Boris Weed gesprochen oder ihn erwähnt? Sie wollte von ihm ein Sack voll Vogelfutter kaufen.", "timestamp": datetime(2025, 1, 5)},
    }

    result = {}
    for key, value in org_data.items():
        if key.startswith(name.lower()): # and min_daterange_ts <= value["timestamp"] <= max_daterange_ts:
            result[key] = value

    if not result:
        return "No information available for this timerange"

    return result

get_messages_tool = FunctionTool.from_defaults(
    fn=get_messages,
    name="get_messages",
)


In [ ]:
tools = [
    find_person_tool,
    find_orgnization_tool,
    get_messages_tool,
    knowledge_tool,
]


In [ ]:
import nest_asyncio

nest_asyncio.apply()

prod_data = ["Die Pilz Mafia ist eine Organisation.", "Die Polnische Polizei überwacht die Pilz Mafia."]

org_result = agent.run_sync(f'What organizations are mentioned {prod_data}')
print(org_result.data)
print(org_result.usage())
#memory.save(org_result.data)


In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.core.memory import ChatMemoryBuffer, ChatSummaryMemoryBuffer

memory = ChatMemoryBuffer(token_limit=2000)
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context, tool_choice='auto',max_iterations=25, timeout=3000, memory=memory) #, chat_history=memory)

# update agent system prompt
react_system_prompt = PromptTemplate(react_system_header_str)
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})
agent.reset()
print("start prompt")
#prompt = "Who is Anna Gölding and what other person may be related to her? to which organzations may she be related?"
question_context = """context: c1, zeitbereich: 2025-02-01T00:00:00+00:00 to 2025-02-15T00:00:00+00:00"""
prompt = f"""Wer war Anna Gölding und welche andren personen oder organisationen stehen mit ihr in verbindung?"""
xprompt = f"""Wer war Anna Gölding und welche andren personen oder organisationen stehen mit ihr in verbindung?
             Liste alle informationen und fakten die du findest in der antwort auf.
            questions context: {question_context}
            1. Analysiere die Person bekannt ist im find person tool.
            2. Analysiere die Person verbindungen zu anderen Personen oder Organisationen hat
            3. Prüfe ob Nachriten (messages) im context dieser Personen im gesuchten Zeitbereich statgefunden haben mit dem get_messages tool.
            4. Nenne die Anzahl der conversationen
            5. Nenne die Teilnehmer der conversationen
            6. Fasse den Inhalt der Kommunikation zusammen
            7. Prüfe ob Entitäten wie Personen, Organisationen oder Orte in der Nachrichten vorkommen, die bisher nicht bekannt sind.
            8. Kontrolliere ob alle Punkte dieser liste erfüllt sind
          """
response = agent.query(prompt)
#response = llm.complete(prompt)

#memory.save_context({"input": prompt}, {"output": str(response)})

print(f"AI: {response}")